In [1]:
%reset -sf

In [42]:
from pprint import pprint
from collections import defaultdict, OrderedDict, namedtuple, deque
from random import choice, seed, randint, shuffle
from tqdm import tqdm
from itertools import repeat, product, combinations, cycle
from pathlib import Path
from string import ascii_lowercase

import numpy as np
import pandas as pd

"""from torch import optim
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torch import (load, amax as pt_amax, max as pt_max, ones, save, no_grad, stack, numel, tensor, 
                   manual_seed, sigmoid, tanh, add, mul, sub, div, amin as pt_amin, cat,
                  maximum, minimum, device, cuda, rand, prod, median, log as pt_log, round as pt_round,
                  isnan, flatten, mean)
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam
#from torchviz import make_dot
import torchvision
import torchvision.transforms as transforms"""

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from deap import creator as ga_cr, base as ga_b, algorithms as ga_algo, tools as ga_t

'from torch import optim\nfrom torch.utils.data import DataLoader, Dataset\nfrom torchvision.datasets import ImageFolder\nfrom torch import (load, amax as pt_amax, max as pt_max, ones, save, no_grad, stack, numel, tensor, \n                   manual_seed, sigmoid, tanh, add, mul, sub, div, amin as pt_amin, cat,\n                  maximum, minimum, device, cuda, rand, prod, median, log as pt_log, round as pt_round,\n                  isnan, flatten, mean)\nfrom torch import nn\nfrom torch.nn import functional as F\nfrom torch.optim import Adam\n#from torchviz import make_dot\nimport torchvision\nimport torchvision.transforms as transforms'

In [43]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [44]:
path = Path('/kaggle') / 'input' / 'hashcode-2021-oqr-extension'
files = list(path.glob('*'))
files

[PosixPath('/kaggle/input/hashcode-2021-oqr-extension/hashcode.in'),
 PosixPath('/kaggle/input/hashcode-2021-oqr-extension/full_problem_description.pdf')]

In [45]:
def parse_input(input_file):
    with open(input_file) as f:
        arr = f.readlines()
    arr = arr[::-1]

    # first line
    line = arr.pop().split()
    duration, num_inters, num_streets, num_cars, fixed_score = [
        int(x) for x in line]
    # print(duration, num_inters, num_streets, num_cars, fixed_score)

    # parse streets with inters
    streets = {}
    for _ in range(num_streets):
        line = arr.pop().split()
        start = int(line[0])
        end = int(line[1])
        street_name = line[2]
        length = int(line[-1])
        streets[street_name] = {"start": start, "end": end, "length": length}
    # print(f"Unique Streets: {len(streets)}")

    # parse cars paths
    cars_paths = []
    for _ in range(num_cars):
        line = arr.pop().split()
        # [1:] because the first word is the length of the sequence
        sequence = line[1:]
        cars_paths.append(sequence)
    # print(f"Unique Car Paths: {len(cars_paths)}")

    return streets, cars_paths


class MutableValue:
    def __init__(self, val=None):
        self.val = val


Street = namedtuple(
    "Street",
    [
        "id",  # The index of the street
        "start",  # The inters object at the start of the street
        "end",  # The inters object at the end of the street
        "name",  # A str
        "duration",  # The length of the street in seconds
        "driving_cars",  # A dict mapping car ids (int) to remaining seconds
        "waiting_cars",  # A deque of car ids (int)
        "arrival_times",  # A dict mapping car ids (int) to their arrival times
        # A dict mapping car ids (int) to their departure times
        "departure_times"
        # You can compute the seconds that a car
        # was waiting at the end of the street
        # by subtracting the arrival time from the departure time.
    ],
)

Inters = namedtuple(
    "Inters",
    [
        "id",  # The index of the Inters
        "incomings",  # A deque of incoming Street objects
        "outgoings",  # A deque of outgoing Street objects
        # The Street object that currently has a green light.
        # Will be wrapped in a MutableValue with a "val" attribute to allow
        # mutating the value without mutating the namedtuple.
        "green_street",
        # An int representing the total number of waiting cars across all
        # incoming streets of this inters.
        # Will be wrapped in a MutableValue with a "val" attribute to allow
        # mutating the value without mutating the namedtuple.
        "num_waiting_cars",
        # The sum of green times of all incoming streets in the schedule.
        # Will be wrapped in a MutableValue with a "val" attribute to allow
        # mutating the value without mutating the namedtuple.
        "schedule_duration",
        # A list mapping (t mod schedule_duration.val) to the street object
        # that is green at time t.
        "green_street_per_t_mod",  # green_street_per_t_mod: name too long
        # A bool indicating whether the green_street ever needs to be
        # updated during the simulation (i.e., whether the schedule has
        # more than one street).
        # Will be wrapped in a MutableValue with a "val" attribute to allow
        # mutating the value without mutating the namedtuple.
        "needs_updates",
    ],
)

# We only use street indices and inters indices here to allow
# fast deep-copies of a schedule for testing out and reverting modifications.
Schedule = namedtuple(
    "Schedule",
    [
        "i_inters",  # The index of the inters
        "order",  # A list of street ids
        "green_times",  # A dict mapping street ids to green times (seconds)
    ],
)


def parse_input2(input_file_path):
    with open(input_file_path) as f:
        lines = deque(f.readlines())

    # Parse the first line
    total_duration, num_interss, num_streets, num_cars, bonus_points = map(
        int, lines.popleft().split()
    )

    # Create empty interss
    interss = tuple(
        Inters(
            id=i,
            incomings=deque(),
            outgoings=deque(),
            green_street=MutableValue(),
            num_waiting_cars=MutableValue(0),
            green_street_per_t_mod=[],
            schedule_duration=MutableValue(),
            needs_updates=MutableValue(False),
        )
        for i in range(num_interss)
    )

    # Parse the streets
    streets = []
    name_to_street = {}
    for i_street in range(num_streets):
        line = lines.popleft().split()
        start, end = map(int, line[:2])
        name = line[2]
        duration = int(line[3])
        street = Street(
            id=i_street,
            start=interss[start],
            end=interss[end],
            name=name,
            duration=duration,
            driving_cars={},
            waiting_cars=deque(),
            arrival_times={},
            departure_times={},
        )
        name_to_street[name] = street
        interss[start].outgoings.append(street)
        interss[end].incomings.append(street)
        streets.append(street)

    # Parse the paths
    paths = []
    for i_car in range(num_cars):
        line = lines.popleft().split()
        path_length = int(line[0])
        path = line[1:]
        assert len(path) == path_length
        path = deque(name_to_street[name] for name in path)
        paths.append(path)

    return (total_duration, bonus_points, interss,
            streets, name_to_street, paths)

# Parse inputs
streets, cars_paths = parse_input(files[0])
duration, bonus_points, inters, streets2, name_to_i_street, paths = parse_input2(files[0])

In [46]:
def create_inters(streets, cars_paths, cars_paths_order):
    if cars_paths_order == 0:
        cars_paths = reversed(sorted(cars_paths, key=len))
    if cars_paths_order == 1:
        cars_paths = sorted(cars_paths, key=len)
    if cars_paths_order == 2:
        shuffle(cars_paths)

    # shuffle(cars_paths)
    # cars_paths = sorted(cars_paths, key=len)
    inters_streets = defaultdict(set)
    for car_path in cars_paths:
        for street in car_path:
            inters_streets[streets[street]["end"]].add(street)
    # print(f"Unique Intersections: {len(inters_streets)}")

    # # Order inters by lenght (incoming streets)
    # OPT_INTERS_ORDER = trial.suggest_int(
    #     'opt_param_inters_order', 0, 1
    # )
    # if OPT_INTERS_ORDER:
    #     inters_streets = dict(
    #         sorted(inters_streets.items(),
    #                key=(lambda value: len(value[1])),
    #                reverse=True))  # biggest inters first
    # else:
    #     inters_streets = dict(
    #         sorted(inters_streets.items(),
    #                key=(lambda value: len(value[1])),
    #                reverse=False))
    return inters_streets


# NOTE: There are some intersections never reached by cars paths
# So, they should not be accounted for in schedules, otherwise error!


def create_time_fractions(interss,
                          grouping=None,
                          lower_time_fraction=None,
                          upper_time_fraction=None,
                          trial=None
                          ):

    # one light schedule for all inters!
    # OPT_TIME_FRACTION = int(trial.suggest_float(
    #     'opt_time_fraction', lower_time_fraction, upper_time_fraction))
    # inters_sched = defaultdict(dict)
    # for inters in interss:
    #     for j, street in enumerate(interss[inters]):
    #         inters_sched[inters][street] = OPT_TIME_FRACTION

    # each inters gets light
    # each light has same schedule
    # time_fractions = []
    # for i in range(len(interss)):
    #     name = "OPT_TIME_FRACTION_" + str(i)  # + '_' + str(j)
    #     # OPT_TIME_FRACTION = trial.suggest_int(
    #     #     name, lower_time_fraction, upper_time_fraction)
    #     OPT_TIME_FRACTION = int(trial.suggest_float(
    #         name, lower_time_fraction, upper_time_fraction))
    #     time_fractions.append(OPT_TIME_FRACTION)
    # inters_sched = defaultdict(dict)
    # for inters, time_fraction in zip(interss, cycle(time_fractions)):
    #     for j, street in enumerate(interss[inters]):
    #         inters_sched[inters][street] = time_fraction

    # in-between previous and next one

    if grouping:
        time_fractions = []
        for i in range(grouping):
            OPT_TIME_FRACTION = int(trial.suggest_float(
                'OPT_TIME_FRACTION_' + str(i),
                lower_time_fraction,
                upper_time_fraction))
            time_fractions.append(OPT_TIME_FRACTION)
        inters_sched = defaultdict(dict)
        for inters, time_fraction in zip(interss, cycle(time_fractions)):
            for j, street in enumerate(interss[inters]):
                inters_sched[inters][street] = time_fraction

    ###################
    # num_inc_strs = []
    # for inters in interss:
    #     num_inc_strs.append(len(list(interss[inters])))
    # print(f'mean street per inters:', {array(num_inc_strs).mean()})
    ###################

    # each inters gets a light
    # each light gets own schedule
    # very granular!
    # inters_sched = defaultdict(dict)
    # for i, inters in enumerate(interss):
    #     for j, street in enumerate(interss[inters]):
    #         # individual time fractions too costly...
    #         name = "OPT_TIME_FRACTION_" + str(i) + '_' + str(j)
    #         OPT_TIME_FRACTION = int(trial.suggest_float(
    #             name, lower_time_fraction, upper_time_fraction))
    #         inters_sched[inters][street] = OPT_TIME_FRACTION

    return inters_sched


def set_schedules(inters_streets, time_fractions):
    # Set schedules
    schedules = []
    for inters, streets in inters_streets.items():
        cycle = []
        for street in streets:
            # time_fraction = 1  # int(randint(1, 5))
            # time_fraction = 1
            cycle.append([inters, street, time_fractions[inters][street]])
        schedules.append(cycle)
    return schedules


def format_schedules(schedules):
    # Parse solution into submission
    res = []
    res.append([len(schedules)])  # requirement
    for cycle in schedules:
        res.append([cycle[0][0]])  # requirement: inters ID
        res.append([len(cycle)])  # requirement: lenght of inters
        for inters, street, time_fraction in cycle:
            res.append([street, time_fraction])
    res1 = "\n".join(" ".join([str(x) for x in row]) for row in res)
    # with open("submission.csv", "w") as text_file:
    #     text_file.write(res1)
    res2 = [" ".join([str(x) for x in row]) for row in res]
    return res1, res2


def parse_schedules(output_file_path, name_to_street):
    # with open(output_file_path) as f:
    #     lines = f.readlines()
    #     lines = deque(lines)
    # adusted to consume from variables,
    # instead of file path
    lines = deque(output_file_path)
    num_schedules = int(lines.popleft())
    schedules = []
    for i_schedule in range(num_schedules):
        i_inters = int(lines.popleft())
        num_incomings = int(lines.popleft())
        order = []
        green_times = {}
        for i_incoming in range(num_incomings):
            street_name, green_time = lines.popleft().split()
            green_time = int(green_time)
            street = name_to_street[street_name]
            order.append(street.id)
            green_times[street.id] = green_time

        schedule = Schedule(i_inters=i_inters, order=order,
                            green_times=green_times)
        schedules.append(schedule)
    return schedules


def reset(streets, interss):
    # Reinitialize mutable data structures
    for street in streets:
        street.driving_cars.clear()
        street.waiting_cars.clear()
        street.arrival_times.clear()
        street.departure_times.clear()

    for inters in interss:
        inters.green_street.val = None
        inters.num_waiting_cars.val = 0
        inters.green_street_per_t_mod.clear()
        inters.schedule_duration.val = None
        inters.needs_updates.val = False


def grade(schedules, streets, interss, paths,
          total_duration, bonus_points, trial):
    reset(streets, interss)

    # We will consume the deques in the paths list. Save a copy of them
    # for later to reset the paths after the simulation.
    paths_copy = [path.copy() for path in paths]

    # Iterate through the schedules and initialize the interss.
    inters_ids_with_schedules = set()
    for schedule in schedules:
        inters = interss[schedule.i_inters]
        inters_ids_with_schedules.add(inters.id)
        first_street = streets[schedule.order[0]]
        inters.green_street.val = first_street
        inters.needs_updates.val = len(schedule.order) > 1
        schedule_duration = 0
        green_street_per_t_mod = inters.green_street_per_t_mod
        for street_id in schedule.order:
            green_time = schedule.green_times[street_id]
            for _ in range(green_time):
                green_street_per_t_mod.append(streets[street_id])
            schedule_duration += green_time
        inters.schedule_duration.val = schedule_duration

    # inters_ids_with_waiting_cars is restricted to interss
    # with schedules
    inters_ids_with_waiting_cars = set()
    for i_car, path in enumerate(paths):
        street = path.popleft()
        street.waiting_cars.append(i_car)
        if street.end.id in inters_ids_with_schedules:
            inters_ids_with_waiting_cars.add(street.end.id)
        street.end.num_waiting_cars.val += 1

    street_ids_with_driving_cars = set()
    score = 0

    # Main simulation loop
    for t in range(total_duration):

        # Drive across interss
        # Store the ids of interss that
        # don't have waiting cars after this.
        inters_ids_to_remove = set()
        for i_inters in inters_ids_with_waiting_cars:
            inters = interss[i_inters]

            if inters.needs_updates.val:
                # Update the green street
                t_mod = t % inters.schedule_duration.val
                inters.green_street.val = inters.green_street_per_t_mod[t_mod]

            green_street = inters.green_street.val
            waiting_cars = green_street.waiting_cars
            if len(waiting_cars) > 0:
                # Drive across the inters
                waiting_car = waiting_cars.popleft()
                green_street.departure_times[waiting_car] = t
                next_street = paths[waiting_car].popleft()
                next_street.driving_cars[waiting_car] = next_street.duration
                street_ids_with_driving_cars.add(next_street.id)

                inters.num_waiting_cars.val -= 1
                if inters.num_waiting_cars.val == 0:
                    inters_ids_to_remove.add(i_inters)

        inters_ids_with_waiting_cars.difference_update(inters_ids_to_remove)

        # Drive across roads
        # Store the ids of streets that don't have driving cars after this.
        street_ids_to_remove = set()
        for i_street in street_ids_with_driving_cars:
            street = streets[i_street]
            driving_cars = street.driving_cars
            for car in list(driving_cars):
                # Update the "time to live" of this car,
                # i.e. the remaining driving seconds.
                ttl = driving_cars[car]
                ttl -= 1
                if ttl < 0:
                    raise ValueError
                elif ttl == 0:
                    # Reached the end of the street
                    del driving_cars[car]
                    if len(paths[car]) == 0:
                        # FINISH
                        score += bonus_points
                        score += total_duration - t - 1

                        # ##############
                        # Handle pruning based on the intermediate value
                        trial.report(score, t)
                        if trial.should_prune():
                            # Done with the simulation. Restore the paths
                            for i_path in range(len(paths)):
                                paths[i_path] = paths_copy[i_path]
                            raise opt.exceptions.TrialPruned()
                        # ##############

                    else:
                        street.waiting_cars.append(car)
                        street.end.num_waiting_cars.val += 1
                        street.arrival_times[car] = t + 1
                        inters_id = street.end.id
                        if inters_id in inters_ids_with_schedules:
                            inters_ids_with_waiting_cars.add(inters_id)
                else:
                    # The car is still driving on the street
                    driving_cars[car] = ttl
            if len(driving_cars) == 0:
                street_ids_to_remove.add(i_street)
        street_ids_with_driving_cars.difference_update(street_ids_to_remove)

    # We are done with the simulation. Restore the paths.
    for i_path in range(len(paths)):
        paths[i_path] = paths_copy[i_path]

    return score

def write_best_score(score, schedules):
    with open(f"submission_score_{score:,}.csv", "w") as text_file:
        text_file.write(schedules)

In [48]:
# Create intersections
inters_streets = create_inters(streets, cars_paths, 2)

# Create schedules times
time_fractions = create_time_fractions(inters_streets,
                                       OPT_GROUPING,
                                       lower_time_fraction,
                                       upper_time_fraction,
                                       trial)
    
# Schedules
schedules = set_schedules(inters_streets, time_fractions)
schedules1, schedules2 = format_schedules(schedules)

# Evaluation
schedules2 = parse_schedules(schedules2, name_to_i_street)
score = grade(schedules2, streets2, inters, paths,
              duration, bonus_points, trial)

NameError: name 'OPT_GROUPING' is not defined

In [ ]:
write_best_score(score, schedules1)